In [2]:
from wsgiref.simple_server import make_server
from wsgiref.util import request_uri

class serverson():
    def localServerApp(self, environ, start_response):
        status = "200 OK"
        headers = [(
            "Content-type", 
            "text/html; charset=utf-8"
            )]  
        
        start_response(status, headers)

        self.query_url = request_uri(environ)

        return ['teste'.encode()]

    def openAuthorization(self):
        self.redirect_uri = "http://localhost:500"
        i = len(self.redirect_uri) - self.redirect_uri[::-1].find(":")
        port = int(self.redirect_uri[i:])

        _i = self.redirect_uri.find("//") + 2
        i_ = len(self.redirect_uri[_i:]) - len(str(port)) - 1
        host = self.redirect_uri[_i:][:i_]

        server = make_server(host, port, self.localServerApp)
        # Cria o app

        # Seta a flag de reuso do endereço com false

        # Cria o server
        server = make_server("localhost", 500, self.localServerApp)

        # Inicia um try

        server.timeout = None
        server.handle_request()

        #timeout server
        #roda uma vez e pega a request

        #pega a resposta
        r = self.query_url

        print(r)


a = serverson()
a.openAuthorization()

http://localhost:500/testemano


127.0.0.1 - - [18/Apr/2025 17:40:09] "GET /testemano HTTP/1.1" 200 5
